In [1]:
#from rl.environments.AbstractConveyor import AbstractConveyor
import yaml
from stable_baselines import PPO2
from stable_baselines.common.vec_env import DummyVecEnv
import matplotlib.pyplot as plt
import rl.helpers
import cv2
import math
import random
import statistics as sts
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import datetime as dt
import rl
import pathlib
from os.path import join

env_name = 'AbstractConveyor'
subdir = '20201123_1937_B'
path = pathlib.Path().absolute()
results_path = join(path, 'rl', 'trained_models', env_name, subdir)
modelpath = join(results_path, 'best_model.zip')

config_path = join(results_path, 'config.yml')
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)
    print('Config loaded from: \n{}\n'.format(config_path))

# load environment with config variables
env_obj = getattr(rl.environments, env_name)
env = env_obj(config)

#load best model from path
model = PPO2.load(modelpath, env=DummyVecEnv([lambda: env]))
print('Model loaded from:\n{}\n'.format(modelpath))

print('Model Parameters are: \n')
print('\t Queue length:\t\t\t{} \n \t Pipeline length: \t\t{}'.format(env.gtp_buffer_length, env.pipeline_length))
print('\t Amount GtP:\t\t\t{} \n \t Types of order carriers: \t{}'.format(env.amount_of_gtps, env.amount_of_outputs))
print('\t To process orders:\t\t{}/{} \n \t Termination condition: \t{}\n'.format(env.max_items_processed, env.amount_of_gtps*env.gtp_demand_size, env.termination_condition))
env.render_plt()

INFO:tensorflow:TensorFlow


INFO:tensorflow:TensorFlow


Config loaded from: 
D:\Drive\git\RL\rl\trained_models\AbstractConveyor\20201123_1937_B\config.yml

Model loaded from:
D:\Drive\git\RL\rl\trained_models\AbstractConveyor\20201123_1937_B\best_model.zip

Model Parameters are: 

	 Queue length:			7 
 	 Pipeline length: 		10
	 Amount GtP:			2 
 	 Types of order carriers: 	2
	 To process orders:		150/200 
 	 Termination condition: 	3



<Figure size 640x480 with 1 Axes>

In [2]:
#test for number of episodes with RL agent
n_episodes = 50
results = {}
results['rl-agent'] = {}
results['rl-agent']['cycle_count'] = 0
results['rl-agent']['idle_time'] = 0
for episode in range(n_episodes):
    # Run an episode
    state = env.reset()
    done = False
    meta_data = []
    while not done:
        action, _ = model.predict(state, deterministic=False)
        state, reward, done, _ = env.step(action)
        if done:
            results['rl-agent']['cycle_count'] += env.cycle_count
            results['rl-agent']['idle_time'] += sum(env.idle_times_operator.values())
results['rl-agent']['cycle_count'] = results['rl-agent']['cycle_count'] /n_episodes
results['rl-agent']['idle_time'] = results['rl-agent']['idle_time'] /n_episodes

#similarly for the heuristic (FIFo)
results['heuristic'] = {}
results['heuristic']['cycle_count'] = 0
results['heuristic']['idle_time'] = 0

for episode in range(n_episodes):
    print('episode: {} \t steps: \t {}'.format(episode, env.steps), end='\r')
    env.reset()
    while not env.terminate:
        env.do_heuristic_guided_step()
    results['heuristic']['idle_time'] += sum(env.idle_times_operator.values())
    results['heuristic']['cycle_count'] += env.cycle_count
results['heuristic']['cycle_count'] = results['heuristic']['cycle_count'] /n_episodes
results['heuristic']['idle_time'] = results['heuristic']['idle_time'] /n_episodes

#and show result
pd.DataFrame.from_dict(results)

,rl-agent,heuristic
cycle_count,220.18,0.0
idle_time,34.02,0.0
